In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import json
import os

import torch.nn.functional as F

import transformers
import numpy as np

import matplotlib.pyplot as plt
from tqdm import tqdm

import json
from glob import glob

In [ ]:
model_IM_path = # Path to CEL trained model
tokenizer = AutoTokenizer.from_pretrained(model_IM_path)
ds_sample = json.load(open(model_IM_path + "/ds_sample.json"))
model = AutoModelForCausalLM.from_pretrained(model_IM_path, device_map='auto')
letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'α', 'β', 'Γ', 'γ', 'Δ', 'δ', 'ε', 'ζ', 'η', 'θ', 'ι', 'κ', 'Λ', 'λ', 'μ', 'ν', 'Ξ', 'ξ', 'Π', 'π', 'Σ', 'σ', 'τ', 'υ', 'Φ', 'φ', 'χ', 'Ψ', 'ψ', 'Ω', 'ω', 'क', 'ख', 'ग', 'घ', 'च', 'छ', 'ज', 'झ', 'ट', 'ठ', 'ड', 'ढ', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'स', 'ह', 'б', 'в', 'г', 'д', 'ж', 'з', 'к', 'л', 'м', 'н', 'п', 'т', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'и', 'э', 'ю', 'я']

In [61]:
letter_inds = []
alphabet_size = 10
for level in range(6):
    letter_inds.append(letters[level*alphabet_size:(level+1)*alphabet_size])

letter_token_inds = []
for level in range(6):
    letter_token_inds.append([tokenizer.encode(' ' + letter)[1] for letter in letter_inds[level]])

for level in range(6):
    print(letter_inds[level])
    print(letter_token_inds[level])

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
[362, 426, 356, 423, 469, 435, 480, 473, 358, 622]
['K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T']
[735, 445, 386, 452, 507, 393, 1229, 432, 328, 350]
['U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd']
[549, 650, 468, 1630, 816, 1901, 264, 293, 272, 294]
['e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n']
[384, 282, 342, 305, 602, 503, 597, 326, 296, 308]
['o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x']
[297, 281, 2874, 436, 274, 259, 577, 348, 289, 865]
['y', 'z', 'α', 'β', 'Γ', 'γ', 'Δ', 'δ', 'ε', 'ζ']
[379, 1167, 19581, 34318, 85316, 63127, 82263, 70434, 60247, 105092]


In [62]:
new_line_token = ":\n"
sep_token = " ->"

def get_output_inds(input_ids, tokenizer):
    token_ls = [tokenizer.decode(s) for s in input_ids[0]]

    sep_inds = [[]]
    for i, s in enumerate(token_ls):
        if s == new_line_token:
            if len(sep_inds[-1]) > 0:
                sep_inds.append([])
        if s == sep_token:
            sep_inds[-1].append(i)
            sep_inds[-1].append(i+1)

    return sep_inds

In [63]:
input_seqs = []
for k in ["IM_test_DMaskoff"]:
    for s in ds_sample[k]:
        input_seqs.append(s['input'])

In [ ]:
temperature = 1

n_levels = 5

tuple_probs_raw = [[] for _ in range(n_levels)]
tuple_probs_weak = [[] for _ in range(n_levels)]
tuple_probs_strong = [[] for _ in range(n_levels)]

for seq in input_seqs:
    input_ids = tokenizer.encode(seq, return_tensors='pt').to(model.device)
    sep_inds = get_output_inds(input_ids[:, :-1], tokenizer)

    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)

    gt_outputs = input_ids[:, 1:]

    for level, inds in enumerate(sep_inds):

        gt_on_inds = gt_outputs[0][torch.tensor(inds).to(model.device)]

        scores_on_inds = outputs.logits[0][torch.tensor(inds).to(model.device)]
        prob_on_inds_raw = F.softmax(scores_on_inds / temperature, dim=-1)
        gt_prob_raw = prob_on_inds_raw[torch.arange(len(prob_on_inds_raw)), gt_on_inds]
        tuple_prob_raw = gt_prob_raw[::2] * gt_prob_raw[1::2]

        scores_on_inds_weak = scores_on_inds.clone().detach()
        scores_on_inds_weak[:, 128108] = -10000
        prob_on_inds_weak = F.softmax(scores_on_inds_weak / temperature, dim=-1)
        gt_prob_weak = prob_on_inds_weak[torch.arange(len(prob_on_inds_weak)), gt_on_inds]
        tuple_prob_weak = gt_prob_weak[::2] * gt_prob_weak[1::2]

        key_inds = torch.tensor(letter_token_inds[level + 1]).to(model.device)
        scores_on_inds_strong = scores_on_inds.clone().detach()
        scores_on_inds_strong[:, torch.tensor(key_inds)] += 100
        prob_on_inds_strong = F.softmax(scores_on_inds_strong / temperature, dim=-1)
        gt_prob_strong = prob_on_inds_strong[torch.arange(len(prob_on_inds_strong)), gt_on_inds]
        tuple_prob_strong = gt_prob_strong[::2] * gt_prob_strong[1::2]

        tuple_probs_raw[level].extend(tuple_prob_raw.tolist())
        tuple_probs_weak[level].extend(tuple_prob_weak.tolist())
        tuple_probs_strong[level].extend(tuple_prob_strong.tolist())

raw_means = np.mean(tuple_probs_raw, axis=1)
weak_means = np.mean(tuple_probs_weak, axis=1)
strong_means = np.mean(tuple_probs_strong, axis=1)

sampling_raw_1_4 = np.mean(raw_means[:4]) * 100
sampling_raw_5 = np.mean(raw_means[4]) * 100

sampling_weak_1_4 = np.mean(weak_means[:4]) * 100
sampling_weak_5 = np.mean(weak_means[4]) * 100

sampling_strong_1_4 = np.mean(strong_means[:4]) * 100
sampling_strong_5 = np.mean(strong_means[4]) * 100


temperature = 1e-5

n_levels = 5

tuple_probs_raw = [[] for _ in range(n_levels)]
tuple_probs_weak = [[] for _ in range(n_levels)]
tuple_probs_strong = [[] for _ in range(n_levels)]

for seq in input_seqs:
    input_ids = tokenizer.encode(seq, return_tensors='pt').to(model.device)
    sep_inds = get_output_inds(input_ids[:, :-1], tokenizer)

    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)

    gt_outputs = input_ids[:, 1:]

    for level, inds in enumerate(sep_inds):

        gt_on_inds = gt_outputs[0][torch.tensor(inds).to(model.device)]

        scores_on_inds = outputs.logits[0][torch.tensor(inds).to(model.device)]
        prob_on_inds_raw = F.softmax(scores_on_inds / temperature, dim=-1)
        gt_prob_raw = prob_on_inds_raw[torch.arange(len(prob_on_inds_raw)), gt_on_inds]
        tuple_prob_raw = gt_prob_raw[::2] * gt_prob_raw[1::2]

        scores_on_inds_weak = scores_on_inds.clone().detach()
        scores_on_inds_weak[:, 128108] = -10000
        prob_on_inds_weak = F.softmax(scores_on_inds_weak / temperature, dim=-1)
        gt_prob_weak = prob_on_inds_weak[torch.arange(len(prob_on_inds_weak)), gt_on_inds]
        tuple_prob_weak = gt_prob_weak[::2] * gt_prob_weak[1::2]

        key_inds = torch.tensor(letter_token_inds[level + 1]).to(model.device)
        scores_on_inds_strong = scores_on_inds.clone().detach()
        scores_on_inds_strong[:, torch.tensor(key_inds)] += 100
        prob_on_inds_strong = F.softmax(scores_on_inds_strong / temperature, dim=-1)
        # print(prob_on_inds_strong.sum(dim=-1))
        gt_prob_strong = prob_on_inds_strong[torch.arange(len(prob_on_inds_strong)), gt_on_inds]
        tuple_prob_strong = gt_prob_strong[::2] * gt_prob_strong[1::2]

        # print(tuple_prob_strong - tuple_prob_weak)

        tuple_probs_raw[level].extend(tuple_prob_raw.tolist())
        tuple_probs_weak[level].extend(tuple_prob_weak.tolist())
        tuple_probs_strong[level].extend(tuple_prob_strong.tolist())

            
raw_means = np.mean(tuple_probs_raw, axis=1)
weak_means = np.mean(tuple_probs_weak, axis=1)
strong_means = np.mean(tuple_probs_strong, axis=1)

greedy_raw_1_4 = np.mean(raw_means[:4]) * 100
greedy_raw_5 = np.mean(raw_means[4]) * 100

greedy_weak_1_4 = np.mean(weak_means[:4]) * 100
greedy_weak_5 = np.mean(weak_means[4]) * 100

greedy_strong_1_4 = np.mean(strong_means[:4]) * 100
greedy_strong_5 = np.mean(strong_means[4]) * 100

print("None:\t ${:.2f}\%$ & ${:.2f}\%$ & ${:.2f}\%$ & ${:.2f}\%$".format(greedy_raw_1_4, greedy_raw_5, sampling_raw_1_4, sampling_raw_5))
print("Weak:\t ${:.2f}\%$ & ${:.2f}\%$ & ${:.2f}\%$ & ${:.2f}\%$".format(greedy_weak_1_4, greedy_weak_5, sampling_weak_1_4, sampling_weak_5))
print("Strong:\t ${:.2f}\%$ & ${:.2f}\%$ & ${:.2f}\%$ & ${:.2f}\%$".format(greedy_strong_1_4, greedy_strong_5, sampling_strong_1_4, sampling_strong_5))

/tmp/ipykernel_1495107/1355119543.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  scores_on_inds_strong[:, torch.tensor(key_inds)] += 100
/tmp/ipykernel_1495107/1355119543.py:101: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  scores_on_inds_strong[:, torch.tensor(key_inds)] += 100


None:	 $0.26\%$ & $2.05\%$ & $0.05\%$ & $1.13\%$
Weak:	 $0.33\%$ & $2.05\%$ & $0.07\%$ & $1.13\%$
Strong:	 $2.11\%$ & $2.05\%$ & $1.91\%$ & $1.34\%$
